In [ ]:
import numpy as np
from log_hazard_rx_binary import grx_dnn_bin
from log_hazard_rx_binary import grax_dnn_bin
from log_hazard_rx_binary import grax1_dnn_bin
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep
from data_generator import gendata_Deep_mixG
from sklearn.model_selection import train_test_split
import torch

In [17]:
patiences = 20
n_node = 64
n_layer = 2
t_nodes = 100
t_fig = 50
n_lr = 5e-4
n_epoch = 1000
s_k = 0.1*np.arange(100)
set_seed(114)
test_data = gendata_Deep_mixG(500, 0.5,pat=3)
seed = 42
rho1 = 1; rho2 = 0

set_seed(1156)
val_data = gendata_Deep_mixG(200, 0.5,pat=3)
train_data = gendata_Deep_mixG(800, 0.5,pat=3)

In [18]:
## Step 1: Data splitting
X1, X2, R1, R2, A1, A2 = train_test_split(train_data['X'],\
                                            train_data['R'], train_data['A'], \
                                            test_size=0.5, random_state=seed) 
train_data1 = {'X':X1, 'A':A1, 'R':R1}
train_data2 = {'X':X2, 'A':A2, 'R':R2}

Xv1, Xv2, Rv1, Rv2, Av1, Av2 = train_test_split(val_data['X'],\
                                            val_data['R'], val_data['A'], \
                                            test_size=0.5, random_state=seed) 
val_data1 = {'X':Xv1, 'A':Av1, 'R':Rv1}
val_data2 = {'X':Xv2, 'A':Av2, 'R':Rv2}
### sample size
n = len(R1)
print(n)

400


In [19]:
## Step 2: Estimate g(r,a,x) and g(r,x)
### train with A
Srax = grax_dnn_bin(train_data1, val_data1, test_data, t_nodes,\
                        t_fig, s_k, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
S_R_A_X = Srax['S_R_X']   ## S(r|a,x)
lam_R_A_X = Srax['Lambda_R_X']   ##lambda(r|a,x)
model_R_A_X = Srax['model']
### train without A
Srx = grx_dnn_bin(train_data2, val_data2, test_data, t_nodes,\
                    t_fig, s_k, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
S_R_X = Srx['S_R_X']    ## S(r|x)
lam_R_X = Srx['Lambda_R_X']    ## lambda(r|x)
model_R_X = Srx['model']

In [20]:
## Step 3:  Calculate the direction function h(r,a,x) and h(r,x)
h_R_A_X = (S_R_A_X**rho1)*((1-S_R_A_X)**rho2)
h_R_X = (S_R_X**rho1)*((1-S_R_X)**rho2)

In [21]:
S_R_A_X_train = Srax['S_R_X_train']
S_R_X_train = Srx['S_R_X_train']

In [23]:
h_R_A_X = (S_R_A_X**rho1)*((1-S_R_A_X)**rho2)
h_R_X = (S_R_X**rho1)*((1-S_R_X)**rho2)
h_R_A_X_train = (S_R_A_X_train**rho1)*((1-S_R_A_X_train)**rho2)
h_R_X_train = (S_R_X_train**rho1)*((1-S_R_X_train)**rho2)

In [24]:
## Step 4: Test statistic and variance
Tn = np.sqrt(n)*(h_R_A_X*(np.log(lam_R_A_X)-np.log(lam_R_X))).mean()

X_train1 = torch.tensor(train_data1['X'], dtype=torch.float32)
A_train1 = torch.tensor(train_data1['A'], dtype=torch.float32)
R_0_train1 = torch.tensor(train_data1['R'], dtype=torch.float32)
X_train2 = torch.tensor(train_data2['X'], dtype=torch.float32)
A_train2 = torch.tensor(train_data2['A'], dtype=torch.float32)
R_0_train2 = torch.tensor(train_data2['R'], dtype=torch.float32)
print(R_0_train1.shape)


### calculate the hazard function
def batch_func_train_R_X(t):
    # t: [batch_size, n_points]
    # A_train = A
    batch_size, n_points = t.shape
    t = t.reshape(-1, 1)  # [batch_size * n_points, 1]
    X_repeated = X_train2.repeat_interleave(n_points, dim=0)  # [batch_size * n_points, d_X]
    inputs = torch.cat((t, X_repeated), dim=1)  # [batch_size * n_points, d_X + 1]
    outputs = model_R_X(inputs).squeeze()  # [batch_size * n_points]
    # outputs2 = 
    # outputs = outputs1*outputs2
    return torch.exp(outputs).reshape(batch_size, n_points)  # 恢复为 [batch_size, n_points]

def batch_func_train_R_A_X(t,A):
    # t: [batch_size, n_points]
    A_train1 = A
    batch_size, n_points = t.shape
    t = t.reshape(-1, 1)  # [batch_size * n_points, 1]
    X_repeated = X_train1.repeat_interleave(n_points, dim=0)  # [batch_size * n_points, d_X]
    A_repeated = A_train1.repeat_interleave(n_points, dim=0)  # [batch_size * n_points, 2]
    inputs = torch.cat((t, X_repeated), dim=1)  # [batch_size * n_points, d_X + 1]
    outputs = model_R_A_X(inputs).squeeze()  # [batch_size * n_points]
    outputs1 = torch.sum(outputs*A_repeated, dim = 1)
    return torch.exp(outputs1).reshape(batch_size, n_points)  # 恢复为 [batch_size, n_points]

torch.Size([400])


In [25]:
n_points = 100
nodes, weights = np.polynomial.legendre.leggauss(n_points)
nodes = torch.tensor(nodes, dtype=torch.float32) 
weights = torch.tensor(weights, dtype=torch.float32) 
b = R_0_train2
a = torch.zeros_like(R_0_train2)
nodes = 0.5 * (nodes + 1) * (b - a).unsqueeze(1) + a.unsqueeze(1)
### the log-hazard results
values_R_X = batch_func_train_R_X(nodes) # n*n_points

nodes1, weights1 = np.polynomial.legendre.leggauss(n_points)
nodes1 = torch.tensor(nodes1, dtype=torch.float32) 
weights1 = torch.tensor(weights1, dtype=torch.float32) 
b1 = R_0_train1
a1 = torch.zeros_like(R_0_train1)
nodes1 = 0.5 * (nodes1 + 1) * (b1 - a1).unsqueeze(1) + a1.unsqueeze(1)
values_R_A_X = batch_func_train_R_A_X(nodes1,A=A_train1)  # n*n_points

In [26]:
### the survival-based direction: h_R_X_train, construct the integral int_0^R exp(g(r,X))h(r,X)dr 
h_R_X_train = torch.tensor(h_R_X_train, dtype=torch.float32) 
# print(h_R_X_train)
h_R_A_X_train = torch.tensor(h_R_A_X_train, dtype=torch.float32) 
intgral_R_X_index = torch.exp(values_R_X)*h_R_X_train   # n*n_points
int_expRX_hX = torch.sum(weights*intgral_R_X_index,dim=1)  # n*1
intgral_R_A_X_index = torch.exp(values_R_A_X)*h_R_X_train   # n*n_points
int_expRAX_hX = torch.sum(weights*intgral_R_A_X_index,dim=1)  # n*1
h_R_X = torch.tensor(h_R_X, dtype=torch.float32) 
h_R_A_X = torch.tensor(h_R_A_X, dtype=torch.float32) 


In [32]:
### score function and variance
Phi_R_X_h = h_R_X - int_expRX_hX #n*1
Phi_R_A_X_h = h_R_X - int_expRAX_hX #n*1
sigmasq_R_X_h = torch.var(Phi_R_X_h)
sigmasq_R_A_X_h = torch.var(Phi_R_A_X_h)
sigmasq_R_X_h = sigmasq_R_X_h.detach().numpy()
sigmasq_R_A_X_h = sigmasq_R_A_X_h.detach().numpy()
sigmasq_Tn_h = 2*(sigmasq_R_X_h+sigmasq_R_X_h)
sigma_Tn_h = np.sqrt(sigmasq_Tn_h)

In [33]:
# Step 5: Decision Making and p-value calculation
import scipy.stats as stats
alpha = 0.05
Un = np.abs(Tn/sigma_Tn_h)
norm_dist = stats.norm(loc=0, scale=1)  #N(0,1)
criteria = norm_dist.ppf(1 - alpha/2)
p_value = 2 * norm_dist.sf(abs(Un))
decision = (Un>criteria).astype(int)

In [34]:
print(p_value)
print(decision)
print(Un)
print(sigma_Tn_h)
print(Tn)
print(int_expRAX_hX.mean())
print(values_R_A_X.max())
print(values_R_X.max())

# print(sigmasq_R_X_h)

0.04863899379475618
1
1.9717426360087118
0.54527307
1.0751381516456604
tensor(0.7865, grad_fn=<MeanBackward0>)
tensor(3.3938, grad_fn=<MaxBackward1>)
tensor(2.1617, grad_fn=<MaxBackward1>)


In [36]:
print(Phi_R_A_X_h.mean())
print(Phi_R_X_h.mean())

tensor(-0.2954, grad_fn=<MeanBackward0>)
tensor(0.0100, grad_fn=<MeanBackward0>)


In [38]:
print(Phi_R_A_X_h.std())
print(torch.sqrt(torch.mean(Phi_R_A_X_h**2)))
print(Phi_R_X_h.std())
print(torch.sqrt(torch.mean(Phi_R_X_h**2)))
print(Tn)

tensor(0.8673, grad_fn=<StdBackward0>)
tensor(0.9152, grad_fn=<SqrtBackward0>)
tensor(0.2726, grad_fn=<StdBackward0>)
tensor(0.2725, grad_fn=<SqrtBackward0>)
1.0751381516456604


In [4]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep_con
from data_generator import gendata_Deep
import torch
# from log_hazard_binary import g_dnn_bin
# from log_linear_binary import E_ols_bin
# from optITR import optITR_lin_bin
from itrtest import confound_test_bin
patiences = [30,20]
n_node = [64,64]
n_layer = [1,1]
t_nodes = 100
t_fig = 50
n_lr = [1e-4,2e-4]
n_epoch = [1000,1000]
s_k = 0.1*np.arange(100)
seed = 42
rho1 = 1; rho2 = 0
loop = 200
decision = np.zeros(loop)
p_value_record = np.zeros(loop)
Un_record = np.zeros(loop)
Tn_record = np.zeros(loop)
sigma_record = np.zeros(loop)


set_seed(114)
test_data = gendata_Deep(500, 0.5) # no use

for i in range(loop):
    set_seed(1899+i)
    # set_seed(1897)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Deep(400, 0.5)
    train_data = gendata_Deep(1600, 0.5)
    res = confound_test_bin(train_data, val_data, test_data,\
                       t_nodes, t_fig, s_k, n_layer, n_node, n_lr, n_epoch,\
                          patiences,rho1=1,rho2=0,seed=42,\
                              show_val = False, n_points=100, alpha=0.05)
    decision[i] = res['decision']
    p_value_record[i] = res['p_value']
    Un_record[i] = res['U_n']
    Tn_record[i] = res['statistic']
    sigma_record[i] = res['sigma_Tn_h']
     
    # time.sleep(1)

print('Result show:       ')
print(f"Decision: {np.mean(decision)}")
print(f"p_value: {np.mean(p_value_record)}")
print(f"Phi_mean: {res['Phi_mean']}")
print(f"Phi_A_mean: {res['Phi_A_mean']}")

Result show:       
Decision: 0.05
p_value: 0.5271610643049784
Phi_mean: 0.31357064843177795
Phi_A_mean: 0.24049098789691925


In [5]:
print('Result show:       ')
print(f"Decision: {np.mean(decision)}")
print(f"p_value: {np.mean(p_value_record)}")
print(f"Phi_mean: {res['Phi_mean']}")
print(f"Phi_A_mean: {res['Phi_A_mean']}")
print(decision)
print(p_value_record)
print(res)

Result show:       
Decision: 0.05
p_value: 0.5271610643049784
Phi_mean: 0.31357064843177795
Phi_A_mean: 0.24049098789691925
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
[0.88392019 0.23447757 0.80050951 0.72076269 0.83050707 0.08096192
 0.28787637 0.32405778 0.89078068 0.89794598 0.6162092  0.78579084
 0.03720582 0.26079172 0.90167457 0.23910852 0.45145587 0.32218424
 0.65723762 0.73540883 0.97757986 0.57402611 0.85863947 0.402431

In [7]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep_con
from data_generator import gendata_Deep
import torch
# from log_hazard_binary import g_dnn_bin
# from log_linear_binary import E_ols_bin
# from optITR import optITR_lin_bin
from itrtest import confound_test_bin
patiences = [30,20]
n_node = [64,64]
n_layer = [1,1]
t_nodes = 100
t_fig = 50
n_lr = [2e-4,2e-4]
n_epoch = [1000,1000]
s_k = 0.1*np.arange(100)
seed = 42
rho1 = 1; rho2 = 0
loop = 40
decision = np.zeros(loop)
p_value_record = np.zeros(loop)
Un_record = np.zeros(loop)
Tn_record = np.zeros(loop)
sigma_record = np.zeros(loop)


set_seed(114)
test_data = gendata_Deep_con(500, 0.5) # no use

for i in range(loop):
    set_seed(1899+i)
    # set_seed(1897)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Deep_con(200, 0.5)
    train_data = gendata_Deep_con(800, 0.5)
    res = confound_test_bin(train_data, val_data, test_data,\
                       t_nodes, t_fig, s_k, n_layer, n_node, n_lr, n_epoch,\
                          patiences,rho1=1,rho2=0,seed=42,\
                              show_val = False, n_points=100, alpha=0.05)
    print(f"Decision: {res['decision']}, p_value: {res['p_value']}")
    decision[i] = res['decision']
    p_value_record[i] = res['p_value']
    Un_record[i] = res['U_n']
    Tn_record[i] = res['statistic']
    sigma_record[i] = res['sigma_Tn_h']
     
    # time.sleep(1)

print('Result show:       ')
print(f"Decision: {np.mean(decision)}")
print(f"p_value: {np.mean(p_value_record)}")
print(f"Phi_mean: {res['Phi_mean']}")
print(f"Phi_A_mean: {res['Phi_A_mean']}")

Decision: 1, p_value: 0.002679126639058225
Decision: 1, p_value: 0.016536229718217582
Decision: 1, p_value: 0.023719950555122427
Decision: 1, p_value: 0.0006642506413568034
Decision: 0, p_value: 0.10036723810662936
Decision: 1, p_value: 2.372988244767353e-06
Decision: 1, p_value: 1.6569200777924804e-05
Decision: 1, p_value: 0.00014580107076084416
Decision: 1, p_value: 0.009242094512332516
Decision: 0, p_value: 0.15916198325329467
Decision: 1, p_value: 0.03156578297427577
Decision: 1, p_value: 4.226326723770842e-06
Decision: 1, p_value: 0.032691724326705945
Decision: 0, p_value: 0.5257140584796669
Decision: 1, p_value: 3.82482089071153e-05
Decision: 0, p_value: 0.669494579283634
Decision: 1, p_value: 1.50671106183197e-06
Decision: 1, p_value: 0.004901405572326012
Decision: 1, p_value: 0.0005611157634933771
Decision: 1, p_value: 5.547921431129143e-05
Decision: 1, p_value: 1.2322821237739421e-05
Decision: 0, p_value: 0.13600121621243594
Decision: 0, p_value: 0.6398152713900623
Decision: 0

In [3]:
print('Result show:       ')
print(f"Decision: {np.mean(decision)}")
print(f"p_value: {np.mean(p_value_record)}")
print(f"Phi_mean: {res['Phi_mean']}")
print(f"Phi_A_mean: {res['Phi_A_mean']}")
print(decision)
print(p_value_record)
print(res)

Result show:       
Decision: 0.7
p_value: 0.11814197766382599
Phi_mean: 0.05458066985011101
Phi_A_mean: -0.8910751938819885
[1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1.]
[2.01536989e-05 1.00347102e-03 5.70255250e-01 9.33271608e-09
 9.22481141e-02 5.73887536e-22 5.74227079e-02 6.22982422e-11
 1.48300551e-10 1.25703137e-03 1.11631632e-01 4.93843284e-06
 1.25437385e-03 3.62565322e-01 5.45318025e-05 1.59018275e-06
 1.79789462e-02 1.24836611e-09 1.48294728e-09 8.38701518e-01
 1.24808296e-11 1.50082722e-01 1.40815777e-09 2.02127050e-03
 8.77914599e-07 1.26552652e-03 4.99702191e-08 3.49112733e-01
 6.65749908e-24 1.36083852e-06 9.53204279e-01 8.45520030e-03
 1.15923823e-14 2.13440321e-02 5.91576905e-02 3.26257178e-02
 5.31591281e-08 9.79305789e-01 8.33378465e-02 3.13643626e-02]
{'statistic': np.float64(1.3805343529028278), 'sigma_Tn_h': np.float32(0.64138734), 'U_n': np.float64(2.1524190757650916), 'criteria': np.flo

In [6]:
print((train_data['R']).max())
print((val_data['R']).max())

17.363073
13.259615


In [7]:
print('Result show:       ')
print(f"Decision: {np.mean(decision)}")
print(f"p_value: {np.mean(p_value_record)}")
print(f"Phi_mean: {res['Phi_mean']}")
print(f"Phi_A_mean: {res['Phi_A_mean']}")
print(decision)
print(p_value_record)
print(res)

Result show:       
Decision: 0.12
p_value: 0.38293431320253646
Phi_mean: 0.07078301161527634
Phi_A_mean: -0.4634135365486145
[0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0.]
[1.57861567e-01 3.56947152e-01 8.42961073e-01 3.11235643e-02
 8.32822748e-01 1.27080011e-02 4.22893129e-01 9.16792614e-03
 8.70162916e-03 1.49029002e-01 7.15273033e-01 1.72046115e-01
 3.31185849e-01 8.46299290e-01 2.37057799e-01 8.77348283e-02
 9.05567542e-01 9.46

In [5]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep_mixG
from data_generator import gendata_Deep
from data_generator import gendata_Deep_size
import torch
# from log_hazard_binary import g_dnn_bin
# from log_linear_binary import E_ols_bin
# from optITR import optITR_lin_bin
from itrtest import confound_test1_bin
patiences = [30,30]
n_node = [64,64]
n_layer = [1,1]
t_nodes = 100
t_fig = 50
n_lr = [5e-4,5e-4]
n_epoch = [1000,1000]
# s_k = 0.1*np.arange(100)
seed = 44
rho1 = 1; rho2 = 0
loop = 20
decision = np.zeros(loop)
p_value_record = np.zeros(loop)
Un_record = np.zeros(loop)
Tn_record = np.zeros(loop)
sigma_record = np.zeros(loop)


set_seed(114)
test_data = gendata_Deep_size(500, 0.5) # no use
# beta = np.array([0,-0.5,0.5,-0.25,0.5])
# gamma = np.array([0.5,-0.25,0,0.5,-0.25])
# set_seed(114)
# test_data = gendata_Linear(500, 0.5,beta,gamma,pat=1)

for i in range(loop):
    set_seed(1999+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Deep_size(400, 0.5)
    train_data = gendata_Deep_size(1600, 0.5)
    s_k = np.linspace(start=0, stop=np.floor(np.max(train_data['R']))+1,num=100)
    res = confound_test1_bin(train_data, val_data, test_data,\
                       t_nodes, t_fig, s_k, n_layer, n_node, n_lr, n_epoch,\
                          patiences,rho1=rho1,rho2=rho2,seed=seed,\
                              show_val =False, n_points=100, alpha=0.05)
    print(f"Decision: {res['decision']}, p_value: {res['p_value']}")
    decision[i] = res['decision']
    p_value_record[i] = res['p_value']
    Un_record[i] = res['U_n']
    Tn_record[i] = res['statistic']
    sigma_record[i] = res['sigma_Tn_h']

print('Result show:       ')
print(f"Decision: {np.mean(decision)}")
print(f"p_value: {np.mean(p_value_record)}")
print(f"Phi_mean: {res['Phi_mean']}")
print(f"Phi_A_mean: {res['Phi_A_mean']}")
# print(f"Decision result: {decision}")
# print(f"P_value result: {p_value_record}")

Decision: 0, p_value: 0.08897371889303235
Decision: 1, p_value: 0.011577635201887516
Decision: 0, p_value: 0.05224579953407179
Decision: 0, p_value: 0.5613849224685776
Decision: 0, p_value: 0.8725591290178951
Decision: 0, p_value: 0.42867395610399894
Decision: 1, p_value: 0.0023723783658035373
Decision: 0, p_value: 0.1472555124938179
Decision: 1, p_value: 0.000416623509195615
Decision: 0, p_value: 0.06400242106622242
Decision: 0, p_value: 0.7481661106130304
Decision: 0, p_value: 0.8078595231171971
Decision: 0, p_value: 0.19909012224893508
Decision: 0, p_value: 0.49857362469528754
Decision: 0, p_value: 0.18980472685963712
Decision: 0, p_value: 0.1685683120318161
Decision: 0, p_value: 0.6124368142121435
Decision: 0, p_value: 0.31962916023825616
Decision: 0, p_value: 0.23926861329689453
Decision: 0, p_value: 0.5934899666466229
Result show:       
Decision: 0.15
p_value: 0.3303174535307162
Phi_mean: -0.039660193026065826
Phi_A_mean: -0.04573187977075577


In [18]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep_mixG
from data_generator import gendata_Deep
from data_generator import gendata_Deep_size
from data_generator import gendata_Deep_ax
import torch
# from log_hazard_binary import g_dnn_bin
# from log_linear_binary import E_ols_bin
# from optITR import optITR_lin_bin
from itrtest import confound_test1_bin
patiences = [30,30]
n_node = [64,64]
n_layer = [1,1]
t_nodes = 100
t_fig = 50
n_lr = [9e-5,9e-5]
n_epoch = [1000,1000]
# s_k = 0.1*np.arange(100)
seed = 42
rho1 = 0; rho2 = 0
loop =200
decision = np.zeros(loop)
p_value_record = np.zeros(loop)
Un_record = np.zeros(loop)
Tn_record = np.zeros(loop)
sigma_record = np.zeros(loop)


set_seed(114)
test_data = gendata_Deep_ax(500, 0.5, rho=0) # no use
# beta = np.array([0,-0.5,0.5,-0.25,0.5])
# gamma = np.array([0.5,-0.25,0,0.5,-0.25])
# set_seed(114)
# test_data = gendata_Linear(500, 0.5,beta,gamma,pat=1)

for i in range(loop):
    set_seed(1999+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Deep_ax(400, 0.5, rho=0)
    train_data = gendata_Deep_ax(1600, 0.5, rho=0)
    s_k = np.linspace(start=0, stop=np.floor(np.max(train_data['R']))+1,num=100)
    res = confound_test1_bin(train_data, val_data, test_data,\
                       t_nodes, t_fig, s_k, n_layer, n_node, n_lr, n_epoch,\
                          patiences,rho1=rho1,rho2=rho2,seed=seed,\
                              show_val =False, n_points=100, alpha=0.05)
    print(f"Decision: {res['decision']}, p_value: {res['p_value']}")
    decision[i] = res['decision']
    p_value_record[i] = res['p_value']
    Un_record[i] = res['U_n']
    Tn_record[i] = res['statistic']
    sigma_record[i] = res['sigma_Tn_h']

print('Result show:       ')
print(f"Decision: {np.mean(decision)}")
print(f"p_value: {np.mean(p_value_record)}")
# print(f"Phi_mean: {res['Phi_mean']}")
# print(f"Phi_A_mean: {res['Phi_A_mean']}")
print(f"Tn: {Tn_record}")
print(f"sigma: {sigma_record}")

Decision: 0, p_value: 0.8252715477748881
Decision: 0, p_value: 0.3629871655239242
Decision: 0, p_value: 0.612619901370411
Decision: 0, p_value: 0.8157109190578244
Decision: 0, p_value: 0.480205025802244
Decision: 0, p_value: 0.8263824011131562
Decision: 0, p_value: 0.5111623520670032
Decision: 0, p_value: 0.892162672023506
Decision: 0, p_value: 0.6547357149317742
Decision: 0, p_value: 0.2053669453405388
Decision: 0, p_value: 0.9946924539429564
Decision: 0, p_value: 0.39523592656539464
Decision: 0, p_value: 0.731563128165003
Decision: 0, p_value: 0.6153884508444041
Decision: 0, p_value: 0.32474986087441293
Decision: 0, p_value: 0.9345093799942576
Decision: 0, p_value: 0.2853494429137666
Decision: 0, p_value: 0.12415454697558184
Decision: 0, p_value: 0.5399268891595941
Decision: 0, p_value: 0.38792134368365905
Decision: 0, p_value: 0.12496142479972447
Decision: 0, p_value: 0.7256764769705435
Decision: 0, p_value: 0.70461399887172
Decision: 0, p_value: 0.8139970962513873
Decision: 0, p_va

In [12]:
print(np.std(Tn_record))
print(np.mean(Tn_record))
print(np.mean(sigma_record))

0.30869834194120205
0.31966015573125334
0.4203338152170181


In [1]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Linear_ax
from data_generator import gendata_Deep_mixG
from data_generator import gendata_Deep
from data_generator import gendata_Deep_size
from data_generator import gendata_Deep_ax
import torch
# from log_hazard_binary import g_dnn_bin
# from log_linear_binary import E_ols_bin
# from optITR import optITR_lin_bin
from itrtest import confound_test1_bin
patiences = [30,30]
n_node = [64,64]
n_layer = [1,1]
t_nodes = 100
t_fig = 50
n_lr = [1e-4,1e-4]
n_epoch = [1000,1000]
# s_k = 0.1*np.arange(100)
seed = 42
rho1 = 1; rho2 = 0
loop = 50
decision = np.zeros(loop)
p_value_record = np.zeros(loop)
Un_record = np.zeros(loop)
Tn_record = np.zeros(loop)
sigma_record = np.zeros(loop)
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])


set_seed(114)
test_data = gendata_Linear_ax(500, 0.5, beta, gamma, rho=0) # no use
# beta = np.array([0,-0.5,0.5,-0.25,0.5])
# gamma = np.array([0.5,-0.25,0,0.5,-0.25])
# set_seed(114)
# test_data = gendata_Linear(500, 0.5,beta,gamma,pat=1)

for i in range(loop):
    set_seed(1925+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear_ax(400, 0.5, beta, gamma, rho=0)
    train_data = gendata_Linear_ax(1600, 0.5, beta, gamma, rho=0)
    s_k = np.linspace(start=0, stop=np.floor(np.max(train_data['R']))+1,num=100)
    res = confound_test1_bin(train_data, val_data, test_data,\
                       t_nodes, t_fig, s_k, n_layer, n_node, n_lr, n_epoch,\
                          patiences,rho1=rho1,rho2=rho2,seed=seed,\
                              show_val =False, n_points=100, alpha=0.05)
    print(f"Decision: {res['decision']}, p_value: {res['p_value']}")
    decision[i] = res['decision']
    p_value_record[i] = res['p_value']
    Un_record[i] = res['U_n']
    Tn_record[i] = res['statistic']
    sigma_record[i] = res['sigma_Tn_h']

print('Result show:       ')
print(f"Decision: {np.mean(decision)}")
print(f"p_value: {np.mean(p_value_record)}")
print(f"Tn: {Tn_record}")
print(f"Sigma: {p_value_record}")

Decision: 0, p_value: 0.846887984688326
Decision: 0, p_value: 0.1095075684381111
Decision: 0, p_value: 0.6928269487327071
Decision: 0, p_value: 0.32439364853454944
Decision: 0, p_value: 0.3548908185892422
Decision: 0, p_value: 0.7036820260343355
Decision: 0, p_value: 0.920321963398234
Decision: 0, p_value: 0.19445558696830822
Decision: 0, p_value: 0.42084385829979487
Decision: 1, p_value: 0.004217829875315713
Decision: 0, p_value: 0.9807355394917912
Decision: 0, p_value: 0.8462190992765919
Decision: 0, p_value: 0.9429250779798077
Decision: 0, p_value: 0.43197218430873985
Decision: 0, p_value: 0.5359815244850441
Decision: 0, p_value: 0.3822317451980708
Decision: 0, p_value: 0.9676963187053931
Decision: 0, p_value: 0.08662000134131437
Decision: 0, p_value: 0.4716781217856185
Decision: 0, p_value: 0.09355676675228795
Decision: 0, p_value: 0.30131446149358343
Decision: 0, p_value: 0.906803140171801
Decision: 0, p_value: 0.5723359057064762
Decision: 0, p_value: 0.4249323216374743
Decision: 

In [3]:
result_data = list['decision': decision,
                   'p_value': p_value_record,
                   'Tn': Tn_record,
                   'sigma': sigma_record,
                   'Un': Un_record
                   ]

np.save('result_linear.npy', result_data)
print('OK')

OK


In [5]:
load_result = np.load('result_linear.npy',allow_pickle=True)
print(load_result)

list[slice('decision', array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]), None), slice('p_value', array([0.84688798, 0.10950757, 0.69282695, 0.32439365, 0.35489082,
       0.70368203, 0.92032196, 0.19445559, 0.42084386, 0.00421783,
       0.98073554, 0.8462191 , 0.94292508, 0.43197218, 0.53598152,
       0.38223175, 0.96769632, 0.08662   , 0.47167812, 0.09355677,
       0.30131446, 0.90680314, 0.57233591, 0.42493232, 0.349685  ,
       0.18203141, 0.80019539, 0.72444014, 0.05707067, 0.11853788,
       0.76117657, 0.35123631, 0.23106135, 0.49703859, 0.43076836,
       0.30569385, 0.02602961, 0.28131047, 0.01060334, 0.22552341,
       0.80263363, 0.30512841, 0.74225967, 0.20602955, 0.42364943,
       0.76481627, 0.3303815 , 0.43109599, 0.78986867, 0.02948715]), None), slice('Tn', array([-0.09267589,  0.81996821,  0.187

In [2]:
print(decision)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1.]


In [13]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep_con
from data_generator import gendata_Deep
from data_generator import gendata_Deep_size
import torch
# from log_hazard_binary import g_dnn_bin
# from log_linear_binary import E_ols_bin
# from optITR import optITR_lin_bin
from itrtest import confound_test1_bin
patiences = [30,30]
n_node = [64,64]
n_layer = [1,1]
t_nodes = 100
t_fig = 50
n_lr = [1e-4,1e-4]
n_epoch = [1000,1000]
# s_k = 0.1*np.arange(100)
seed = 42
rho1 = 1; rho2 = 0
loop =10
decision = np.zeros(loop)
p_value_record = np.zeros(loop)
Un_record = np.zeros(loop)
Tn_record = np.zeros(loop)
sigma_record = np.zeros(loop)


set_seed(114)
test_data = gendata_Deep(500, 0.5) # no use
# beta = np.array([0,-0.5,0.5,-0.25,0.5])
# gamma = np.array([0.5,-0.25,0,0.5,-0.25])
# set_seed(114)
# test_data = gendata_Linear(500, 0.5,beta,gamma,pat=1)

for i in range(loop):
    set_seed(1999+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Deep(400, 0.5)
    train_data = gendata_Deep(1600, 0.5)
    s_k = np.linspace(start=0, stop=np.floor(np.max(train_data['R']))+1,num=100)
    res = confound_test1_bin(train_data, val_data, test_data,\
                       t_nodes, t_fig, s_k, n_layer, n_node, n_lr, n_epoch,\
                          patiences,rho1=rho1,rho2=rho2,seed=seed,\
                              show_val =False, n_points=100, alpha=0.05)
    print(f"Decision: {res['decision']}, p_value: {res['p_value']}")
    decision[i] = res['decision']
    p_value_record[i] = res['p_value']
    Un_record[i] = res['U_n']
    Tn_record[i] = res['statistic']
    sigma_record[i] = res['sigma_Tn_h']

print('Result show:       ')
print(f"Decision: {np.mean(decision)}")
print(f"p_value: {np.mean(p_value_record)}")
print(f"Phi_mean: {res['Phi_mean']}")
print(f"Phi_A_mean: {res['Phi_A_mean']}")

Decision: 1, p_value: 0.0190296728004368
Decision: 1, p_value: 0.015110430107161238
Decision: 0, p_value: 0.12689196209252007
Decision: 0, p_value: 0.05098025629922378
Decision: 0, p_value: 0.07031340424789737
Decision: 1, p_value: 0.023554460261263857
Decision: 0, p_value: 0.2894147573620931
Decision: 1, p_value: 0.005376519878077609
Decision: 0, p_value: 0.06100124246407064
Decision: 1, p_value: 0.01028690033495221
Result show:       
Decision: 0.5
p_value: 0.06719596058476966
Phi_mean: -0.07496597617864609
Phi_A_mean: -0.10583856701850891
